In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model_id = "/data/qm/huggingface/Mistral-7B-Instruct-v0.2/Mistral-7B-Instruct-v0.2"

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant",
     "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
generated_ids = model.generate(model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
2024-05-10 01:25:52.710953: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 01:25:54.061976: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


"<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Yes, I do have a simple and classic mayonnaise recipe for you! Here's what you'll need:\n\n* 1 egg yolk\n* 1 teaspoon Dijon mustard\n* 1 tablespoon white wine vinegar or fresh lemon juice\n* 1 cup vegetable oil (canola or sunflower oil works well)\n* Salt, to taste\n\nInstructions:\n\n1. In a medium-sized bowl, whisk"

In [5]:
prompt = '''
Your goal is to split the task given below into a number of subtasks, with the required number of subtasks being no less than two and no more than five.
Please return a JSON object.
The original task and the paths to the files involved in the subtasks should be stored in "File Information".Each file should contain a "file path" field and a "description" field.
The contents of the subtasks should be stored in "Subtasks".Each subtask should contain an 'id' field and a 'task description' field.

For example,
Task: I need to extract the water zone of XiHu district, only keep the water zone
      the green band tif of XiHu district is in $GREEN_BAND_FILEPATH.
      the nir band tif of XiHu district is in $NIR_BAND_FILEPATH.
      please save the final result in $OUTPUT_PATH.

Output:
{{
  "Subtasks":[
    {{
      "id": 1,
      "task description": "Calculate the ndwi of XiHu district."
    }},
    {{
      "id": 2,
      "task description": "Extract the water zone."
    }}
  ],
  "File Information":[
    {{
      "file path": "$GREEN_BAND_FILEPATH",
      "description": "the green band tif of XiHu district."
    }},
    {{
      "file path": "$NIR_BAND_FILEPATH",
      "description": "the nir band tif of XiHu district."
    }},
    {{
      "file path": "$OUTPUT_PATH",
      "description": "Used to save the final result."
    }}
  ]
}}


Task: There are road shapefile without time and speed information, administrative region shapefile without area imformation and hospitals point shapefile.
  Please create service areas for tertiary hospitals within 0-20 minutes and 20-45 minutes driving distance
  and Calculate the percentage of the total area covered by the service areas of top-tier hospitals within 0-20 minutes and 20-45 minutes in each administrative region.

  Here is the data you will need:
  shape_file_path='data/road.shp',
  hospitals_path='data/hospitals.shp',
  administrative_region_path='data/administrative.shp'.

  The meaning of the 'fclass' field for main roads and the average vehicle speed are as follows:
  motorway: 100km/h;
  motoway_link: 40km/h;
  trunk: 70km/h;
  primary: 55km/h;
  primary_link:40km/h;
  secondary_link:40km/h;
  trunk:40k5/h;
  tertiary_link:40km/h;


Begin!
Describe your subtasks with rich details. Do not response anything unrelated to the subtask or file information.
Remeber to ouput a JSON object.
'''
messages = [
    {"role": "user", "content": prompt}
]

model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
generated_ids = model.generate(model_inputs, max_new_tokens=2048, do_sample=True)
result = tokenizer.batch_decode(generated_ids)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [7]:
print(result)

<s> [INST] 
Your goal is to split the task given below into a number of subtasks, with the required number of subtasks being no less than two and no more than five.
Please return a JSON object.
The original task and the paths to the files involved in the subtasks should be stored in "File Information".Each file should contain a "file path" field and a "description" field.
The contents of the subtasks should be stored in "Subtasks".Each subtask should contain an 'id' field and a 'task description' field.

For example,
Task: I need to extract the water zone of XiHu district, only keep the water zone
      the green band tif of XiHu district is in $GREEN_BAND_FILEPATH.
      the nir band tif of XiHu district is in $NIR_BAND_FILEPATH.
      please save the final result in $OUTPUT_PATH.

Output:
{{
  "Subtasks":[
    {{
      "id": 1,
      "task description": "Calculate the ndwi of XiHu district."
    }},
    {{
      "id": 2,
      "task description": "Extract the water zone."
    }}
  ],